# Colab Start Section

Step 1 is to upload a copy of your private key. I am assuming your private key is named id_ed25519. If that isn't the case, you'll have to make manual changes.

In [29]:
# Change these of course to what is your email and your GitHub username
EMAIL = "pauleanderson@gmail.com"
USERNAME = "instructor"

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
%%bash -s "$EMAIL" "$USERNAME"

rm -rf /root/.ssh/*
mkdir /root/.ssh

mv id_ed25519 /root/.ssh/

chmod 600 /root/.ssh/id_ed25519

git config --global user.email "$1"
git config --global user.name "$2"

In [2]:
%%bash -s "$USERNAME"

eval "$(ssh-agent -s)"

ssh-add ~/.ssh/id_ed25519

ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

git clone git@github.com:anderson-fall-2024/csc-448-$1.git

UsageError: Cell magic `%%shell` not found.


# Topic 5

## Clustering Algorithms - Chapter 8

Motivation and some exercises are variations on those available in Bioinformatics Algorithms: An Active-Learning Approach by Phillip Compeau & Pavel Pevzner.

## Slack ice breaker
Best meal you've ever eaten?

## Prelude

There are so many ways we could slice and dice this chapter. Fundamentally, we've got a lot of different angles. We could approach this chapter from a biological and biochemical perspective and focus on the chemsitry and biology necessary to perform transcriptomics. Or we could dive into the statistical approaches necessary to accurately quantify gene expression values. Or we could focus more on alignment algorithms that power the heart of this analysis. We could also focus on the problem beginning at a gene expression values and then focus on algorithms that analyze data. We are going to try to strike a balance in the following order:
1. Discuss some of the biochemistry that makes modern sequencing possible
2. Discuss some of the different ways scientists investigate what is going on inside a cell
3. Discuss the clustering algorithms that are the first lines of the analysis

In [30]:
#### NO NEED TO EDIT ####
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.insert(0,f"csc-448-{USERNAME}")
sys.path.insert(0,"..")

from py448 import Topic5_helper

#### NO NEED TO EDIT ####

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## What's all this about yeast and wine?

The species of yeast that we will consider in this chapter is Saccharomyces cerevisiae. Why?

**It can brew wine because it converts the glucose found in fruit into ethanol**

**Our question:**<br>

If S. cerevisiae often lives on grapevines, **why must crushed grapes be stored in tightly sealed barrels in order to make wine?

* If the supply of glucose runs out, S. cerevisiae must do something to survive
* It will then invert its metabolism, with the ethanol (alcohol) that it just produced becoming its new food supply. 
* This metabolic inversion, called the diauxic shift, can only occur in the presence of oxygen. 
* Without oxygen, S. cerevisiae hibernates until either glucose or oxygen becomes available. 

In conclusion, if winemakers don’t seal their barrels, then the yeast in the barrel will metabolize the ethanol that it just produced, ruining the wine.

The diauxic shift is a complex process that affects the expression of many genes. 

## Our data
>In 1997, Joseph DeRisi conducted the first massive gene expression experiment by sampling an S. cerevisiae culture every two hours for the six hours before and after the diauxic shift. Since there are approximately 6,400 genes in S. cerevisiae, and there were seven time points, this experiment resulted in a 6,400 × 7 gene expression matrix. 

In [31]:
import pandas as pd
df=pd.read_csv('http://bioinformaticsalgorithms.com/data/realdatasets/Clustering/diauxic_raw_ratios_RG.txt',sep='\t')
df

,ORF,Name,R1.Ratio,R2.Ratio,R3.Ratio,R4.Ratio,R5.Ratio,R6.Ratio,R7.Ratio
1,YHR007C,ERG11,1.123596,1.190476,1.315789,0.877193,0.840336,0.383142,0.434783
2,YAL065C,NaN,0.909091,0.653595,1.136364,0.970874,0.781250,1.123596,1.234568
3,YAR062W,NaN,0.751880,1.149425,1.098901,0.925926,0.628931,1.176471,1.351351
4,YDR006C,SOK1,1.111111,0.833333,0.458716,0.334448,0.348432,0.671141,0.546448
5,YDR007W,TRP1,1.041667,1.020408,1.052632,0.787402,0.980392,1.063830,0.952381
...,...,...,...,...,...,...,...,...,...
6149,YDR001C,NTH1,1.098901,1.020408,1.369863,1.818182,1.587302,4.166667,3.703704
6150,YDR002W,YRB1,1.204819,1.265823,1.219512,1.000000,0.892857,1.010101,0.427350
6151,YDR003W,NaN,0.884956,1.785714,0.917431,1.162791,1.123596,2.631579,2.040816
6152,YDR004W,RAD57,0.775194,0.917431,1.298701,0.826446,0.699301,1.587302,1.030928


Values above 1 in expression vectors correspond to increased expression, while values below 1 correspond to decreased expression.

In [12]:
import altair as alt
plot_df = df.set_index('ORF').drop('Name',axis=1).loc[['YPR055W','YLR258W','YPL012W']]
plot_df.columns.name = 'Sample Point'
plot_df = plot_df.stack().to_frame()
plot_df.columns=["Ratio"]
plot_df = plot_df.reset_index()
alt.Chart(plot_df).mark_line().encode(
    x='Sample Point:N',
    y='Ratio',
    color='ORF'
)

alt.Chart(...)

**Stop and think:** What is the interpretation of this plot?

### YOUR SOLUTION HERE

Consider what to do about the other genes?

In [13]:
df.shape

(6153, 9)

**Stop and think:** Considering the dataset above and what you now know about clustering, what questions could you ask?

### YOUR SOLUTION HERE
### YOUR SOLUTION HERE

## Sample of genes

In [14]:
import altair as alt
plot_df = df.set_index('ORF').drop('Name',axis=1).sample(n=100)
plot_df.columns.name = 'Sample Point'
plot_df = plot_df.stack().to_frame()
plot_df.columns=["Ratio"]
plot_df = plot_df.reset_index()
alt.Chart(plot_df).mark_line().encode(
    x='Sample Point:N',
    y='Ratio',
    color='ORF'
)

alt.Chart(...)

**Stop and think:** What are your observations about this graph?

### YOUR SOLUTION HERE
### YOUR SOLUTION HERE

Let's remove genes that are not of interest. This is done in the textbook by removing genes that don't go up or down by a significant amount. 

In [20]:
df_subset = pd.read_csv('http://bioinformaticsalgorithms.com/data/realdatasets/Clustering/230genes_log_expression.txt',sep='\t')
df_subset

,ORF,Name,R1.Ratio,R2.Ratio,R3.Ratio,R4.Ratio,R5.Ratio,R6.Ratio,R7.Ratio
1,YDR025W,RPS18A,0.136062,-0.111031,-0.189034,-0.782409,-0.757023,-0.855990,-2.304511
2,YDR031w,NaN,-0.286881,-0.084064,0.184425,0.136062,0.535332,2.321928,1.251539
3,YDR060w,NaN,-0.042644,-0.097611,-0.014355,-0.799087,-0.839960,-2.247928,-2.386811
4,YDR064W,YS15,-0.056584,-0.124328,-0.070389,-0.545968,-0.555816,-2.104337,-2.367371
5,YDR070c,NaN,0.014500,0.074001,0.058894,0.014500,0.251539,2.395929,1.689660
...,...,...,...,...,...,...,...,...,...
226,YDL085w,NaN,-0.111031,1.000000,0.058894,-0.124328,-0.275007,2.058894,2.836501
227,YDL136w,NaN,-0.070389,-0.014355,-0.042644,-0.367371,-0.214125,-1.550901,-2.563158
228,YDL199c,NaN,-0.042644,0.494109,0.415037,0.184425,-0.344828,1.736966,2.321928
229,YDL204w,NaN,-0.367371,0.915936,-0.505891,0.268817,0.089267,4.058894,3.058894


### Redo the plot

In [16]:
plot_df = df_subset.set_index('ORF').drop('Name',axis=1).sample(n=100)
plot_df.columns.name = 'Sample Point'
plot_df = plot_df.stack().to_frame()
plot_df.columns=["Ratio"]
plot_df = plot_df.reset_index()
alt.Chart(plot_df).mark_line().encode(
    x='Sample Point:N',
    y='Ratio',
    color='ORF'
)

alt.Chart(...)

**Stop and think:** Now that you know about k-means clustering, what is a good $k$ value?

In [7]:
# our standard imports
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans

**Exercise 1:** Using your k value, cluster the genes using k-means. You may use sklearn's version of kmeans. Color the plot above using your clusters.

In [22]:
df_subset.drop(['ORF','Name'],axis=1)

,R1.Ratio,R2.Ratio,R3.Ratio,R4.Ratio,R5.Ratio,R6.Ratio,R7.Ratio
1,0.136062,-0.111031,-0.189034,-0.782409,-0.757023,-0.855990,-2.304511
2,-0.286881,-0.084064,0.184425,0.136062,0.535332,2.321928,1.251539
3,-0.042644,-0.097611,-0.014355,-0.799087,-0.839960,-2.247928,-2.386811
4,-0.056584,-0.124328,-0.070389,-0.545968,-0.555816,-2.104337,-2.367371
5,0.014500,0.074001,0.058894,0.014500,0.251539,2.395929,1.689660
...,...,...,...,...,...,...,...
226,-0.111031,1.000000,0.058894,-0.124328,-0.275007,2.058894,2.836501
227,-0.070389,-0.014355,-0.042644,-0.367371,-0.214125,-1.550901,-2.563158
228,-0.042644,0.494109,0.415037,0.184425,-0.344828,1.736966,2.321928
229,-0.367371,0.915936,-0.505891,0.268817,0.089267,4.058894,3.058894


In [24]:
clusterer = KMeans(n_clusters=2, random_state=10)
df_subset["Cluster"] = clusterer.predict(df_subset.drop(['ORF','Name'],axis=1))
df_subset

,ORF,Name,R1.Ratio,R2.Ratio,R3.Ratio,R4.Ratio,R5.Ratio,R6.Ratio,R7.Ratio,Cluster
1,YDR025W,RPS18A,0.136062,-0.111031,-0.189034,-0.782409,-0.757023,-0.855990,-2.304511,0
2,YDR031w,NaN,-0.286881,-0.084064,0.184425,0.136062,0.535332,2.321928,1.251539,1
3,YDR060w,NaN,-0.042644,-0.097611,-0.014355,-0.799087,-0.839960,-2.247928,-2.386811,0
4,YDR064W,YS15,-0.056584,-0.124328,-0.070389,-0.545968,-0.555816,-2.104337,-2.367371,0
5,YDR070c,NaN,0.014500,0.074001,0.058894,0.014500,0.251539,2.395929,1.689660,1
...,...,...,...,...,...,...,...,...,...,...
226,YDL085w,NaN,-0.111031,1.000000,0.058894,-0.124328,-0.275007,2.058894,2.836501,1
227,YDL136w,NaN,-0.070389,-0.014355,-0.042644,-0.367371,-0.214125,-1.550901,-2.563158,0
228,YDL199c,NaN,-0.042644,0.494109,0.415037,0.184425,-0.344828,1.736966,2.321928,1
229,YDL204w,NaN,-0.367371,0.915936,-0.505891,0.268817,0.089267,4.058894,3.058894,1


**Problem 2:** Plot all of the genes with color according to their cluster. 

In [26]:
### YOUR SOLUTION HERE
### YOUR SOLUTION HERE

alt.Chart(...)

**Stop and think:** How can you now if you selected the right number of clusters?

### YOUR SOLUTION HERE
### YOUR SOLUTION HERE

**Exercise 2:** Analyze two clusterings (k=2 and k=3) by calculating the silhouette score.

In [27]:
from sklearn.metrics import silhouette_score

### YOUR SOLUTION HERE
cluster2 = clusterer2.predict(df_subset.drop(['ORF','Name','Cluster'],axis=1))
cluster3 = clusterer3.predict(df_subset.drop(['ORF','Name','Cluster'],axis=1))
print('Score for k=2',silhouette_score(df_subset.drop(['ORF','Name','Cluster'],axis=1), cluster2))
print('Score for k=3',silhouette_score(df_subset.drop(['ORF','Name','Cluster'],axis=1), cluster3))

Score for k=2 0.7544866254183358
Score for k=3 0.518441778993325
